In [1]:
import sail.core
import json
import getpass

password = "SailPassword@123"

sail.core.login("nadams@mghl.com", password, 1234, "apiservices")
print(sail.core.get_datasets())
print(sail.core.get_digital_contracts())

{
    "42A85F64-5717-4562-B3FC-2C963F66AFA6": {
        "keywords": "string",
        "tables": [
            {
                "description": "string",
                "compressed_data_size_in_bytes": 0.000000,
                "all_column_properties": [
                    {
                        "description": "string",
                        "type": "string",
                        "tags": "string",
                        "name": "string",
                        "units": "string",
                        "id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
                    }
                ],
                "number_of_columns": 0.000000,
                "name": "string",
                "data_size_in_bytes": 0.000000,
                "tags": "string",
                "number_of_rows": 0.000000,
                "id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
            }
        ],
        "name": "DS",
        "organization": {
            "name": "International Genetics Research F

In [2]:
#########################################################
# Allocate our job and load our SafeObjects
#########################################################

sail.core.load_safe_objects("../SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
dg_sum_column_guid = "442D0290DEA14CE4B70FB31B891F20B6"

job_id = sail.core.run_job(dg_test_object_guid)
print("Job ID: {} ".format(job_id))

Job ID: None 


In [2]:
#########################################################
# Extract our identifiers
#########################################################
json_datasets = json.loads(sail.core.get_datasets())
json_digital_contracts = json.loads(sail.core.get_digital_contracts())

user_dataset_name = input("Enter name of Dataset you uploaded:" ) or "SanityDataset"
print("The dataset you wish to access is: {}".format(user_dataset_name))

provision_dc = ""
provision_ds = ""
my_table_guid = ""
for dataset in json_datasets:
    if(json_datasets[dataset]["name"] == user_dataset_name):
        provision_ds = dataset.lower()
        my_table_guid = json_datasets[dataset]["tables"][0]["id"]

for digital_contract in json_digital_contracts:
    if (json_digital_contracts[digital_contract]["dataset"]["id"] == provision_ds):
        provision_dc = digital_contract.lower()
    
print("DC {} DS {} TABLE {}".format(provision_dc, provision_ds, my_table_guid))

Enter name of Dataset you uploaded:DS
The dataset you wish to access is: DS
DC 38ca0bde-82e9-4561-a44d-8d6282b1d90c DS 42a85f64-5717-4562-b3fc-2c963f66afa6 TABLE 3fa85f64-5717-4562-b3fc-2c963f66afa6


In [4]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

TypeError: argument 1 must be str, not None

In [3]:
first_provision = sail.core.provision_secure_computational_node("orch-test02", provision_dc, provision_ds, "STANDARD_B4MS")

print("Provision ID: {}".format(first_provision))


Provision ID: {
    "Status": true,
    "SCNGuid": "e768136d-d7f3-432a-a6a3-1719b867dfcd"
}


In [10]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(45000))


{
    "Failed": {

    },
    "Succeeded": {
        "E768136D-D7F3-432A-A6A3-1719B867DFCD": ""
    },
    "InProgress": {

    },
    "AllDone": true
}


In [ ]:
############################################################################
#
# Set the parameters for the job and set a variable for its output
#
############################################################################

set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")
print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))

In [ ]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")

#pull_data_result = sail.core.pull_data(data_id)
#print(pull_data_result)

In [ ]:
############################################################################
#
# Produce data frame from CSV
#
############################################################################

import base64
import pickle

message_response = sail.core.wait_for_data(10000)
data_frame = None;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Dataframe has been processed from CSV")
            data_frame = un_serialized_value
            
    message_response = sail.core.wait_for_data(2000)


In [ ]:
############################################################################
#
# Get the sum of a column
#
############################################################################
sum_column_job_id = sail.core.run_job(dg_sum_column_guid)

column_name = "BMI"
# Setup a job to injest a dataframe, and sum a column's name
pushed_column_name = sail.core.push_user_data(column_name)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4C3F2D549B2042E291C47A074A0194A6", data_id)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4C275957B5C34C20BD0E02E03330167D", pushed_column_name)
print("Column job IP {}".format(sail.core.get_ip_for_job(sum_column_job_id)))
column_result_id = "{}.{}".format(sum_column_job_id, "5036707EAABB46C584FC5DBA75D5718E")

pull_data_result = sail.core.pull_data(column_result_id)

In [ ]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(20000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("The sum of column {} is {}".format(column_name, un_serialized_value))
            
    message_response = sail.core.wait_for_data(5000)

In [6]:
sail.core.deprovision_secure_computational_node('9e803511-6326-4a58-a79c-05c8cd60fc52')

0

In [ ]:
sail.core.exit_current_session()